# Calculation 02 - Elastic constants
The following notebook will calculate the elastic constants of Nickel-based EAM and MEAM potentials. The first cell needs updating depending on the potential file you are using and the compound you want to run the calculation for.\
At the end of the notebook, there is a "fast run" which will allow you to run this notebook for all potentials and all phases in one go.

## 📌 Specify filenames and element related data
For any potential, please specify the filename of the potential file as follows:\
`potentials/filename.eam`\
`potentials/filename.eam.fs`\
`potentials/filename.eam.alloy`\
`potentials/filename.meam`

Then uncomment the respective lines, depending on which compound you want to run the calculation for.

**Note:** to call Non-LAMMPS variables within a LAMMPS command, please use an f-string. To call LAMMPS variables, instead of using an f-string, just use curly braces with a leading dollar sign - like this: `${x}`.

In [ ]:
potentialpath = "potentials/Ni1986.eam"

##### insert whichelements, crystalstructure, latticeconstant and outputfile here #####


'''
#fcc-Ni
compound = "Ni"
whichelements = "Ni"
crystalstructure = "fcc"
latticeconstant = 3.52
outputfile = "data/02_elastic-constants_Ni.dat"

#B2-NiAl
compound = "NiAl"
whichelements = "Ni Al"
crystalstructure = "bcc"
latticeconstant = 2.86
outputfile = "data/02_elastic-constants_NiAl.dat"

#L12-Ni3Al
compound = "Ni3Al"
whichelements = "Ni Al"
crystalstructure = "fcc"
latticeconstant = 3.56
outputfile = "data/02_elastic-constants_Ni3Al.dat"
'''

## ⬇️ Import PyLammps module

In [ ]:
from lammps import PyLammps
lmp = PyLammps()

lmp.clear()

## ⬇️ Add parameters for MEAM potentials
There is no need to change the following cell. It defines two parameters that are necessary for reading in MEAM potentials.\
**Please do note that the libraryelements variable has to contain the element symbols in the same order as they appear in the library file!**

In [ ]:
if "meam" in potentialpath:
    libraryfile = str(potentialpath.replace('.meam', '.library.meam'))

    if "NiAlCo2" in potentialpath:
        libraryelements = "Ni Al Co"
    elif "NiAl2" in potentialpath:
        libraryelements = "Al Ni"
    else:
        libraryelements = "Ni"
else:
    libraryfile = 0
    libraryelements = 0

## ⬇️ Initialize simulation
`units` sets the unit system for this simulation. "metal" units are Angstroms for distance, eV for energy, etc.\
`dimension` is self-explanatory and can either be 2 or 3.\
`boundary` sets the boundary conditions in x-, y- and z-direction. Here, they are periodic in all three dimensions.\
`atom_style` specifies additional attributes depending on the style - "atomic" doesn't have any.

In [ ]:
lmp.units('metal')
lmp.dimension('3')
lmp.boundary('p p p')
lmp.atom_style('atomic')

## ⬇️ Create atoms
`lattice` specifies the predefined or customized lattice and the respective lattice constant.\
`region` specifies the simulation cell as a geometric shape, like a sphere, cylinder or block. The default coordinate unit is a "lattice parameter", meaning each coordinate is an integer factor times the lattice parameter specified in the `lattice` command. "cell" is the user-assigned ID for this specific region, but could also be called "1", "a" or anything else. For periodic boundary conditions, the cell dimensions should be multiples of the lattice constant to prevent unwanted overlaps (as explained in the documentation for the `create_atoms` command).\
`create_box` actually creates the simulation cell as defined in the `region` command. The integer defines the number of element types that will be used, the string defines the region in which the cell will be created.\
`create_atoms` adds atoms to the lattice within a specific region ("box" fills the entire simulation cell defined in the `region` command with atoms), so this has to happen after defining and creating the simulation box. The integer defines the number of element types used. "basis" allows to assign coordinates to atom types: the first integer is the coordinate (they can be found in the documentation for the `lattice` command), the second one is the element index in the same order as it will be specified when reading in the potentials (Ni = 1, Al = 2).
`replicate` would allow to change the size of the simulation, and in this position is equivalent to changing the coordinates in the `region` command.
`mass` does not need to specified for any EAM or MEAM potential since it is used from the potential file itself.

In [ ]:
lmp.lattice(f'{crystalstructure} {latticeconstant}')
lmp.region('cell prism 0 5 0 5 0 5 0 0 0')

if compound == "Ni":
    lmp.create_box('1 cell')
    lmp.create_atoms('1 box')
elif compound == "NiAl":
        lmp.create_box('2 cell')
        lmp.create_atoms('2 box basis 1 1 basis 2 2')
elif compound == "Ni3Al":
        lmp.create_box('2 cell')
        lmp.create_atoms('2 box basis 1 2 basis 2 1 basis 3 1 basis 4 1')

## ⬇️ Define function: Read in potential file 
`pair_style` specifies what kind of interatomic potential will be used.\
`pair_coeff` specifies the potential file to be read in. The input parameters change depending on the potential style. that the pair potential coefficients are stored in. The first two integers define the force field coefficients between atom pairs. The asterisks include all atom types.\
`neighbor` and `neigh_modify` would set parameters for the neighbor lists - the default parameters when the commands are not used can be found in the LAMMPS documentation.

In [ ]:
def potential(libraryfile, libraryelements, potentialpath, whichelements):
    if "meam" in potentialpath:
        lmp.pair_style('meam')
        lmp.pair_coeff(f'* * {libraryfile} {libraryelements} {potentialpath} {whichelements}')
    elif "eam.alloy" in potentialpath:
        lmp.pair_style('eam/alloy')
        lmp.pair_coeff(f'* * {potentialpath} {whichelements}')
    elif "eam.fs" in potentialpath:
        lmp.pair_style('eam/fs')
        lmp.pair_coeff(f'* * {potentialpath} {whichelements}')
    else:
        lmp.pair_style('eam')
        lmp.pair_coeff(f'* * {potentialpath}')

## ⬇️ Read in potential file

In [ ]:
potential(libraryfile, libraryelements, potentialpath, whichelements)

## ⬇️ Define thermo settings
Storing the input parameters for the energy minimization as the `minimize` command is used multiple times in this script.\
`min_style` would set which minimization algorithm should be used for the minimization. The default value is `cg` - conjugate gradient.\
`min_modify` defines the distance an atom can travel during the conjugate gradient energy minimization.\
`thermo` sets the timestep interval at which computes shall be performed and printed during the run.\
`thermo_style` allows to specify what information shall be printed at each of those timesteps - in this case, it is customized to calculate and print the timestep, the length of the simulation box in x, y and z direction, the total pressure, the xx/yy/zz/xy/xz/yz components of the pressure tensor and the total volume of the system.

In [ ]:
lmp.variable('etol equal 0.0') 
lmp.variable('ftol equal 1.0e-10')
lmp.variable('maxiter equal 100')
lmp.variable('maxeval equal 1000')

lmp.variable('up equal 1.0e-6')

In [ ]:
lmp.min_modify('dmax 1.0e-2')

lmp.thermo('10')
lmp.thermo_style('custom step temp pe lx ly lz press pxx pyy pzz pxy pxz pyz vol')

## ⬇️ Define fix
`fix` applies specific operations to a specified group of atoms (in this case, `all` of them). Here, the operation is to relax the entire simulation cell (`box/relax`), using the parameter `aniso 0.0` (pressure in each direction is independent of the other).

In [ ]:
lmp.fix('1 all box/relax aniso 0.0')

## ⬇️ Run minimization
`minimize`s the energy of the simulation cell, using the parameters defined in `fix(1)` and printing out the desired information (as specified in `thermo_style`) at every `thermo(10)`th timestep.

In [ ]:
lmp.minimize('${etol} ${ftol} ${maxiter} ${maxeval}')

## ⬇️ Calculate initial stress tensor components

In [ ]:
pressures = ['pxx', 'pyy', 'pzz', 'pyz', 'pxz', 'pxy']
lengths = ['lx', 'ly', 'lz']

for p in pressures:
    lmp.variable('tmp equal '+p)
    lmp.variable(p+'0 equal ${tmp}')

for l in lengths:
    lmp.variable('tmp equal '+l)
    lmp.variable(l+'0 equal ${tmp}')

for i, p in enumerate(pressures):
    lmp.variable('d'+str(i+1)+' equal -(v_'+p+'1-${'+p+'0})*(1e-4)/(v_delta/v_len0)')

## ⬇️ Displace atoms randomly

In [ ]:
lmp.displace_atoms('all random 1.0e-5 1.0e-5 1.0e-5 100 units box')

## ⬇️ Define unfix
`unfix` terminates previously defined fix. 

In [ ]:
lmp.unfix('1')

## ⬇️ Write equilibrium restart file

In [ ]:
lmp.write_restart('restart.equil')

## ⬇️ Define function: Perform displacements

In [ ]:
def displacement(posneg, dir_ext, libraryfile, libraryelements, potentialpath, whichelements):
    # Reset box and simulation parameters
    lmp.clear()
    lmp.box('tilt large')
    lmp.read_restart('restart.equil')
    
    potential(libraryfile, libraryelements, potentialpath, whichelements)
    
    lmp.min_modify('dmax 1.0e-2')

    lmp.thermo('10')
    lmp.thermo_style('custom step temp pe lx ly lz press pxx pyy pzz pxy pxz pyz vol')
    
    if posneg == "neg":
        prefactor = -1
    elif posneg == "pos":
        prefactor = 1
        
    lmp.variable('delta equal '+str(prefactor)+'*${up}*${len0}')
    lmp.variable('deltaxy equal '+str(prefactor)+'*${up}*xy')
    lmp.variable('deltaxz equal '+str(prefactor)+'*${up}*xz')
    lmp.variable('deltayz equal '+str(prefactor)+'*${up}*yz')
    
    if dir_ext == 1:
        lmp.change_box('all x delta 0 ${delta} xy delta ${deltaxy} xz delta ${deltaxz} remap units box')   
    elif dir_ext == 2:
        lmp.change_box('all y delta 0 ${delta} yz delta ${deltayz} remap units box')
    elif dir_ext == 3:
        lmp.change_box('all z delta 0 ${delta} remap units box')
    elif dir_ext == 4:
        lmp.change_box('all yz delta ${delta} remap units box')
    elif dir_ext == 5:
        lmp.change_box('all xz delta ${delta} remap units box') 
    elif dir_ext == 6:
        lmp.change_box('all xy delta ${delta} remap units box') 

    #Relax atoms positions
    lmp.minimize('${etol} ${ftol} ${maxiter} ${maxeval}')

    #Obtain new stress tensor
    pressures = ['pxx', 'pyy', 'pzz', 'pyz', 'pxz', 'pxy']
    
    for p in pressures:
        lmp.variable('tmp equal '+p)
        lmp.variable(p+'1 equal ${tmp}')

    #Compute elastic constant from pressure tensor            
    for i in range(1,7):
        if posneg == "neg":
            lmp.variable('C'+str(i)+'neg equal ${d'+str(i)+'}')
        elif posneg == "pos":
            lmp.variable('C'+str(i)+'pos equal ${d'+str(i)+'}')

## ⬇️ Perform displacements

In [ ]:
all_len0 = ['lx0', 'ly0', 'lz0', 'lz0', 'lz0', 'ly0']

for i, len0 in enumerate(all_len0):
    dir_ext = i+1
    
    lmp.variable('len0 equal ${'+str(len0)+'}')
    
    displacement("neg", dir_ext, libraryfile, libraryelements, potentialpath, whichelements)
    displacement("pos", dir_ext, libraryfile, libraryelements, potentialpath, whichelements)
        
    for i in range(1,7):
        lmp.variable('C'+str(i)+str(dir_ext)+' equal 0.5*(${C'+str(i)+'neg}+${C'+str(i)+'pos})')

## ⬇️ Calculate properties
This section defines all the components of the stress tensor as `variable`s and then averages those for cubic crystals.

In [ ]:
lmp.variable('c11all equal ${C11}')
lmp.variable('c22all equal ${C22}')
lmp.variable('c33all equal ${C33}')
lmp.variable('c44all equal ${C44}')
lmp.variable('c55all equal ${C55}')
lmp.variable('c66all equal ${C66}')

lmp.variable('c12all equal 0.5*(${C12}+${C21})')
lmp.variable('c13all equal 0.5*(${C13}+${C31})')
lmp.variable('c23all equal 0.5*(${C23}+${C32})')

lmp.variable('C11 equal (${c11all}+${c22all}+${c33all})/3')
lmp.variable('C12 equal (${c12all}+${c13all}+${c23all})/3')
lmp.variable('C44 equal (${c44all}+${c55all}+${c66all})/3')

lmp.variable('bulk equal (${C11}+2*${C12})/3')

lmp.variable('shear_voigt equal (${C11}-${C12}+(3*${C44}))/5')
lmp.variable('shear_reuss equal 5/((4/(${C11}-${C12}))+(3/${C44}))')
lmp.variable('shear_vrh equal (${shear_reuss}+${shear_voigt})/2')

lmp.variable('young equal 9*${bulk}*${shear_vrh}/(3*${bulk}+${shear_vrh})')

lmp.variable('poisson equal (3*${bulk}-2*${shear_vrh})/(6*${bulk}+2*${shear_vrh})')

## ⬇️ Print desired output
This section `print`s these values to the output file, for which you chose a filename in the beginning.

In [ ]:
potentialname = potentialpath.replace('potentials/', '')

lmp.print(f'"{potentialname}'+', ${C11}, ${C12}, ${C44}, ${bulk}, ${shear_vrh}, ${young}, ${poisson}" append '+f'{outputfile} screen no')

🏁**ALL DONE!**🏁

In [ ]:
#please execute the two "define function" cells before starting the fast run!

In [ ]:
def elasticconstants(compound, whichelements, crystalstructure, latticeconstant, outputfile, potentialpath):
    lmp.clear()
        
    if "meam" in potentialpath:
        libraryfile = str(potentialpath.replace('.meam', '.library.meam'))

        if "NiAlCo2" in potentialpath:
            libraryelements = "Ni Al Co"
        elif "NiAl2" in potentialpath:
            libraryelements = "Al Ni"
        else:
            libraryelements = "Ni"
    else:
        libraryfile = 0
        libraryelements = 0
    
    lmp.units('metal')
    lmp.dimension('3')
    lmp.boundary('p p p')
    lmp.atom_style('atomic')
        
    lmp.lattice(f'{crystalstructure} {latticeconstant}')
    lmp.region(f'cell prism 0 5 0 5 0 5 0 0 0')

    if compound == "Ni":
        lmp.create_box('1 cell')
        lmp.create_atoms('1 box')
    elif compound == "NiAl":
            lmp.create_box('2 cell')
            lmp.create_atoms('2 box basis 1 1 basis 2 2')
    elif compound == "Ni3Al":
            lmp.create_box('2 cell')
            lmp.create_atoms('2 box basis 1 2 basis 2 1 basis 3 1 basis 4 1')
    
    potential(libraryfile, libraryelements, potentialpath, whichelements)

    lmp.variable('etol equal 0.0') 
    lmp.variable('ftol equal 1.0e-10')
    lmp.variable('maxiter equal 100')
    lmp.variable('maxeval equal 1000')

    lmp.variable('up equal 1.0e-6')
    
    lmp.min_modify('dmax 1.0e-2')

    lmp.thermo('10')
    lmp.thermo_style('custom step temp pe lx ly lz press pxx pyy pzz pxy pxz pyz vol')

    lmp.fix('1 all box/relax aniso 0.0')

    lmp.minimize('${etol} ${ftol} ${maxiter} ${maxeval}')

    pressures = ['pxx', 'pyy', 'pzz', 'pyz', 'pxz', 'pxy']
    lengths = ['lx', 'ly', 'lz']
    
    for p in pressures:
        lmp.variable('tmp equal '+p)
        lmp.variable(p+'0 equal ${tmp}')
    
    for l in lengths:
        lmp.variable('tmp equal '+l)
        lmp.variable(l+'0 equal ${tmp}')

    for i, p in enumerate(pressures):
        lmp.variable('d'+str(i+1)+' equal -(v_'+p+'1-${'+p+'0})*(1e-4)/(v_delta/v_len0)')

    lmp.displace_atoms('all random 1.0e-5 1.0e-5 1.0e-5 100 units box')

    lmp.unfix('1')

    lmp.write_restart('restart.equil')

    all_len0 = ['lx0', 'ly0', 'lz0', 'lz0', 'lz0', 'ly0']

    for i, len0 in enumerate(all_len0):
        dir_ext = i+1

        lmp.variable('len0 equal ${'+str(len0)+'}')

        displacement("neg", dir_ext, libraryfile, libraryelements, potentialpath, whichelements)
        displacement("pos", dir_ext, libraryfile, libraryelements, potentialpath, whichelements)

        for i in range(1,7):
            lmp.variable('C'+str(i)+str(dir_ext)+' equal 0.5*(${C'+str(i)+'neg}+${C'+str(i)+'pos})')

    lmp.variable('C11all equal ${C11}')
    lmp.variable('C22all equal ${C22}')
    lmp.variable('C33all equal ${C33}')
    lmp.variable('C44all equal ${C44}')
    lmp.variable('C55all equal ${C55}')
    lmp.variable('C66all equal ${C66}')

    lmp.variable('C12all equal 0.5*(${C12}+${C21})')
    lmp.variable('C13all equal 0.5*(${C13}+${C31})')
    lmp.variable('C23all equal 0.5*(${C23}+${C32})')

    lmp.variable('C11cubic equal (${C11all}+${C22all}+${C33all})/3')
    lmp.variable('C12cubic equal (${C12all}+${C13all}+${C23all})/3')
    lmp.variable('C44cubic equal (${C44all}+${C55all}+${C66all})/3')

    lmp.variable('bulkmodulus equal (${C11cubic}+2*${C12cubic})/3')

    lmp.variable('shear_voigt equal (${C11cubic}-${C12cubic}+(3*${C44cubic}))/5')
    lmp.variable('shear_reuss equal 5/((4/(${C11cubic}-${C12cubic}))+(3/${C44cubic}))')
    lmp.variable('shear_vrh equal (${shear_reuss}+${shear_voigt})/2')

    lmp.variable('young equal 9*${bulk}*${shear_vrh}/(3*${bulk}+${shear_vrh})')

    lmp.variable('poisson_poly equal (3*${bulk}-2*${shear_vrh})/(6*${bulk}+2*${shear_vrh})')
    
    potentialname = potentialpath.replace('potentials/', '')

    lmp.print(f'"{potentialname}'+', ${C11cubic}, ${C12cubic}, ${C44cubic}, ${bulk}, ${shear_vrh}, ${young}, ${poisson_poly}" append '+f'{outputfile} screen no')

In [ ]:
results_Ni = []
results_NiAl = []
results_Ni3Al = []

with open('data/01_lattice-constant_Ni.dat') as f:
    for line in f:
        result = [element.strip() for element in line.split(',')]
        results_Ni.append(result)
        
with open('data/01_lattice-constant_NiAl.dat') as f:
    for line in f:
        result = [element.strip() for element in line.split(',')]
        results_NiAl.append(result)
        
with open('data/01_lattice-constant_Ni3Al.dat') as f:
    for line in f:
        result = [element.strip() for element in line.split(',')]
        results_Ni3Al.append(result)

In [ ]:
all_files = []
alloy_files = []

cohesiveenergy_Ni = []
cohesiveenergy_NiAl = []
cohesiveenergy_Ni3Al = []

latticeconstant_Ni = []
latticeconstant_NiAl = []
latticeconstant_Ni3Al = []

for i in range(len(results_Ni)):
    potentialname = results_Ni[i][0]
    cohesiveenergy = results_Ni[i][1]
    latticeconstant = results_Ni[i][2]
    
    all_files.append(potentialname)  
    latticeconstant_Ni.append(latticeconstant)
    
for i in range(len(results_NiAl)):
    potentialname = results_NiAl[i][0]
    cohesiveenergy = results_NiAl[i][1]
    latticeconstant = results_NiAl[i][2]
    
    alloy_files.append(potentialname)
    latticeconstant_NiAl.append(latticeconstant)
    
for i in range(len(results_Ni3Al)):
    potentialname = results_Ni3Al[i][0]
    cohesiveenergy = results_Ni3Al[i][1]
    latticeconstant = results_Ni3Al[i][2]
    
    alloy_files.append(potentialname)
    latticeconstant_Ni3Al.append(latticeconstant)

In [ ]:
compound = ["Ni", "NiAl", "Ni3Al"]
whichelements = ["Ni", "Ni Al", "Ni Al"]
crystalstructure = ["fcc", "bcc", "fcc"]
latticeconstants = [latticeconstant_Ni, latticeconstant_NiAl, latticeconstant_Ni3Al]
filelist = [all_files, alloy_files, alloy_files]

for comp, element, crystal, latticeconstant, files in zip (compound, whichelements, crystalstructure, latticeconstants, filelist):
    for file, lc in zip(files, latticeconstant):
        
        from lammps import PyLammps
        lmp = PyLammps()
        
        potentialpath = "potentials/"+file
        outputfile = "data/02_elastic-properties_"+comp+".dat"
                        
        elasticconstants(comp, element, crystal, lc, outputfile, potentialpath)

🏁**FAST RUN DONE!**🏁